# Bootstrap Metrics ile Değerlendirme

Bootstrap metrikleri, bu not defterinde göreceğimiz gibi, bir ticaret stratejisini daha kapsamlı bir şekilde değerlendirmemize yardımcı olabilir.

[Önceki not defterinde](https://www.pybroker.com/en/latest/notebooks/2.%20Backtesting%20a%20Strategy.html), bir ticaret stratejisi uyguladık ve bunu geriye doğru test ettik. İşte yine uygulama:

In [1]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('my_strategy')

def buy_low(ctx):
    if ctx.long_pos():
        return
    if ctx.bars >= 2 and ctx.close[-1] < ctx.low[-2]:
        ctx.buy_shares = ctx.calc_target_shares(0.25)
        ctx.buy_limit_price = ctx.close[-1] - 0.01
        ctx.hold_bars = 3
        
def short_high(ctx):
    if ctx.short_pos():
        return
    if ctx.bars >= 2 and ctx.close[-1] > ctx.high[-2]:
        ctx.sell_shares = 100
        ctx.hold_bars = 2

Daha önce olduğu gibi, verilen yapılandırmalarla yeni bir [Strategy](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) örneği oluşturuyoruz:

In [2]:
config = StrategyConfig(initial_cash=500_000, bootstrap_sample_size=100)
strategy = Strategy(YFinance(), '3/1/2017', '3/1/2022', config)
strategy.add_execution(buy_low, ['PGSUS.IS', 'EREGL.IS'])
strategy.add_execution(short_high, ['DOAS.IS'])

Bu kez, ```Strateji``` şu şekilde yapılandırılmıştır: [bootstrap_sample_size](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.bootstrap_sample_size) Kullanılan geçmiş veri miktarının az olması nedeniyle ```100```. Daha sonra, önyükleme metrikleri etkinken arka testi çalıştırıyoruz:

In [3]:
result = strategy.backtest(calc_bootstrap=True)
result.metrics_df

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loaded cached bar data.

Test split: 2017-03-01 00:00:00 to 2022-02-28 00:00:00


  0% (0 of 1281) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
 19% (251 of 1281) |####                 | Elapsed Time: 0:00:00 ETA:  00:00:00
 40% (521 of 1281) |########             | Elapsed Time: 0:00:00 ETA:   0:00:00
 62% (801 of 1281) |#############        | Elapsed Time: 0:00:00 ETA:   0:00:00
 82% (1061 of 1281) |################    | Elapsed Time: 0:00:00 ETA:   0:00:00
100% (1281 of 1281) |####################| Elapsed Time: 0:00:00 Time:  0:00:00



Calculating bootstrap metrics: sample_size=100, samples=10000...
Calculated bootstrap metrics: 0:00:04 

Finished backtest: 0:00:06


,name,value
0,trade_count,3.400000e+02
1,initial_market_value,5.000000e+05
2,end_market_value,3.751790e+05
3,total_pnl,-1.248210e+05
4,unrealized_pnl,-1.455192e-11
5,total_return_pct,-2.496420e+01
6,total_profit,2.669176e+05
7,total_loss,-3.917386e+05
8,total_fees,0.000000e+00
9,max_drawdown,-1.519012e+05


Yukarıdaki ```total_pnl``` metriğine baktığımızda, ilk denememizde karlı bir ticaret stratejimiz olduğu görülüyor. Ancak bu sonuçların tekrarlanabilir olduğundan ve sadece şansa bağlı olmadığından tam olarak emin olamayız. Sonuçlarımıza daha fazla güven kazandırmak amacıyla metrikleri hesaplamak için [boostrap yöntemini](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) kullanabiliriz.

Bootstrap yöntemi, arka testin getirilerinden alınan rastgele örnekler üzerinde bir metriği tekrar tekrar hesaplayarak çalışır. Daha sonra, her rastgele örnek için metrik hesaplanır ve ortalaması alınır. Bunu binlerce rastgele örnek üzerinde yaparak, metriğin daha sağlam ve doğru bir tahminini elde ederiz.


## Güvenilirlik aralığı

**PyBroker** iki performans metriği olan [Kar Faktörü](https://www.pybroker.com) için [güven aralıklarını](https://en.wikipedia.org/wiki/Confidence_interval) hesaplamak için önyükleme yöntemini uygular.

In [4]:
result.bootstrap.conf_intervals

lower     upper
name          conf                     
Profit Factor 97.5%  1.210456  6.183094
              95%    1.379809  6.183094
              90%    1.617628  6.183094
Sharpe Ratio  97.5% -0.009397  0.295414
              95%    0.019236  0.295414
              90%    0.054929  0.295414

**PyBroker** hesaplamak için [önyargı düzeltilmiş ve hızlandırılmış (BCa) önyükleme yöntemini](https://blogs.sas.com/content/iml/2017/07/12/bootstrap-bca-interval.html) kullanır. Bu ölçümler için güven aralıkları. Metriklerde daha fazla bilgi elde etmek için getiriler işlem başına değil çubuk başına örneklenir.

Ortaya çıkan tablo, verilen güven düzeyinde güven aralığının alt sınırını gösterir. Bu, stratejinin performansına ilişkin daha ihtiyatlı bir tahmin sağlar. Örneğin, Sharpe Oranının belirli bir *x* değerinde veya üzerinde olduğundan "%97,5"" emin olabiliriz.

Bu örnekte Sharpe Oranının negatif alt sınırları vardır ve Kâr Faktörünün alt sınırları 1'den küçüktür, bu da stratejinin güvenilir olmadığını göstermektedir.

## Maksimum Düşüş

Bu bölümde bootstrap yöntemini kullanarak stratejinin maksimum düşümünü inceliyoruz. Nakit ve portföy özkaynak yüzdesi cinsinden ifade edilen belirli değerleri aşmayan düşüş olasılıkları aşağıda gösterilmektedir:

In [5]:
result.bootstrap.drawdown_conf

,amount,percent
conf,,
99.9%,-71990.28,-15.249940
99%,-60218.27,-12.936398
95%,-47273.53,-10.368205
90%,-41076.93,-9.056090


Bu güven seviyeleri, Kâr Faktörü ve Sharpe Oranının hesaplanmasına benzer şekilde, arka testin örneklem dışı sonuçlarından çubuk başına getiriler kullanılarak elde edildi.

"%99,9" güven seviyesinde "-%10,46" önyüklemeli maksimum düşümün, orijinal sonuçlarımızda gördüğümüz "-%4,55"'ten daha kötü olduğunu gözlemleyebiliriz. Bu, ticaret stratejinizin performansını değerlendirmek için rastgele testler kullanmanın önemini vurgulamaktadır.

[Bir sonraki not defterinde, sıralama ve konum boyutlandırmayı ticaret stratejilerinize nasıl dahil edebileceğinizi tartışacağız](https://www.pybroker.com/en/latest/notebooks/4.%20Ranking%20and%20Position%20Sizing.html ).